# 📈 Predicting the Price of Electricty with Machine Learning with Prophet

![](https://miro.medium.com/max/1400/1*sjUBCoQXy5rfWOfguP_K1A.png)

**The PUN (Italian acronym for Prezzo Unico Nazionale, "National Single Price")** is the wholesale reference price of electricity purchased on the Borsa Elettrica Italiana market (IPEX - Italian Power Exchange). At the Italian Power Exchange, active since 2007 following the entry into force of the Legislative Decree governing the liberalization of the electricity market, the transactions between producers and suppliers of electricity are regulated. The PUN therefore represents the national weighted average of the zonal sales prices of electricity for each hour and for each day. The national figure is an amount that is calculated on the average of various factors, and which takes into account the quantities and prices formed in the different areas of Italy and at different times of the day.


**How the PUN affects the price of energy**
The wholesale price of electricity is established directly on the market based on the trades between the various players involved, i.e. between producers and energy suppliers (who purchase the energy from producers to supply to their end customers). The fluctuations of the PUN are a determining factor in calculating the final costs of energy in the bill. In fact, in the periods in which the PUN increases its value, costs tend to rise, to fall instead when the value of the PUN falls. Energy suppliers generally provide for tariffs for the final consumer at a fixed cost or at an indexed cost as regards the price of the energy component. Opting for an indexed price of the energy component means that this will vary over time depending on the performance of the PUN on the Italian Power Exchange. An offer at a fixed price of the energy component, on the other hand, will remain unchanged for a certain period of time depending on the offer chosen, generally for one or two years.


**The PUN unit measure** is €/MWh 

 https://www.enel.it/en/supporto/faq/cos-e-il-pun

**The following project is about modeling PUN in Italy with Machine Learning algorithms, with the goal of forecasting the hourly PUN for 7 future days. The time series modeling will be performed by Prophet by Facebook and Neural Prophet models**

We have data from January 2016 to June 2022

# Import libraries and loading the data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import calendar
import os
from prophet import Prophet
#from neuralprophet import NeuralProphet

import sklearn
from sklearn import metrics
import math

import datetime
import matplotlib.dates as mdates

plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc("axes", labelweight="bold", labelsize="large", titleweight="bold", titlesize=14, titlepad=10)

In [ ]:
# Input data files are available in the read-only "../input/" directory
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# read file and parse date
df = pd.read_csv("/kaggle/input/energy-pun-main-zones/energy_pun_main_zones.csv",parse_dates = ['DATE'])

In [ ]:
# drop id , we don't need it
df = df.drop(columns = 'IDX')

In [ ]:
# visualize dataframe
df

In [ ]:
# check if there are nulls
df.info()

# Define custom Functions

In [ ]:
def num_to_time(num):
  """ 
  Function convert number to time format
  num: hour as numneric
  """
  time = num
  hours = int(time)
  # hour
  if hours == 24:
    hours = 0
  # minutes and sec
  minutes = 0
  seconds = 0
  out_time = "%02d:%02d:%02d" % (hours, minutes, seconds)
  
  return out_time

In [ ]:
def create_features(df):
    """
    Creates time series features from date column
    df: dataframe
    """
    df = df.copy()
    df['dayofweek'] = df['date'].dt.dayofweek
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.isocalendar().week
    df['dayname'] =  df['date'].dt.day_name()
    # df = df[['dayofweek','month','year','dayofyear','dayofmonth','weekofyear','holidays']]
   
    return df


In [ ]:
#Using plotly.express

def my_plot(df, colum_name, ylabel=None, title=None, start=None, end=None, navigate = 0 ):
  """
  Python function to plot a specific column of dataframe
  Possibiliti to filter: last month, last semester,last year , ytd
  df: dataframe
  colum_name : column to plot
  ylabel:  y axis label
  title: plot title
  start: start date for the plot
  end: end date for the plot
  navigate: give the possibiliti to zoom in and out for the dates range
  """
  if start is not None or end is not None:
    df = df[start:end]
  
  fig = px.line(df, x=df.index, y=colum_name)


  fig.update_layout(
      title= title,
      xaxis_title= "Date",
      yaxis_title= ylabel 
  )   

  if navigate == 1:

      fig.update_xaxes(
          rangeslider_visible=True,
          rangeselector=dict(
              buttons=list([
                  dict(count=1, label="1m", step="month", stepmode="backward"),
                  dict(count=6, label="6m", step="month", stepmode="backward"),
                  dict(count=1, label="YTD", step="year", stepmode="todate"),
                  dict(count=1, label="1y", step="year", stepmode="backward"),
                  dict(step="all")
              ])
          )
      )
  return fig


In [ ]:

def prepare_df_prophet(df,label,date_col,date_index):
    """ 
    Label and feature prepare for prophet 
    label - 
    date_col - 
    date_index - if the the date is an index
    """
    df = df.copy()
    if date_index == True:
      df.reset_index(inplace=True)
    
    df.rename(columns = {label:'y'}, inplace = True)
    df.rename(columns = {date_col:'ds'}, inplace = True)
    df = df[['ds','y']]
    return df    


In [ ]:
def format_fore_test_df(df_test,forecast):
    """ 
    Format prediction and test to evaluate the model performance
    """
    y_pred  = forecast.iloc[-len_test:][['ds','yhat']].copy()
    y_pred = y_pred.set_index('ds')

    # fit the model
    y_test = df_test[['ds','y']].copy()
    y_test.set_index('ds',inplace = True)
    
    return  y_test, y_pred

In [ ]:
def plot_pred_test(y_test,y_pred):
    plt.rcParams["figure.figsize"] = [12, 6]
    ax = plt.gca()
    #ax.xaxis.set_major_locator(mdates.DateLocator)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%d-%m-%Y'))
    plt.gcf().autofmt_xdate(rotation=90) 
    plt.plot(y_test, label='Actual')
    plt.plot(y_pred, label='Predicted')
    plt.legend()
    plt.show()

    return

In [ ]:
# find the first and the last Monday
def from_weekday_to_weekday(df,dayname):
    """
    Takes a dataframe with weekday columns and return the data from the first dayname to the lasta dayname
    in the dataframe

    df: input dataframe 
    dayname: from dayname to dayname
    """
    # inizialize the index
    idx=0 
    
    # first day
    while(True):
        if df_three_months.iloc[idx].dayname == dayname:
            break
        else:
            idx+=1

    idx_min = idx

    # last day
    idx = len (df_three_months) -1 
    
    while(True):
        if df_three_months.iloc[idx].dayname == dayname:
            break
        else:
            idx-=1        

    idx_max = idx   
    #from to
    df =  df.iloc[idx_min:idx_max]
    
    return df

In [ ]:
def error_metrics(y_test, y_pred):
    """
    Calculate the most common forecast error metrics
    y_test: variable name y
    y_pred: variable name yhat
    """
    #R2 - coefficient of determination
    R2 = sklearn.metrics.r2_score(y_test, y_pred)
    
    #Mean squared error
    MSE = sklearn.metrics.mean_squared_error(y_test, y_pred)
    
    #Root Mean squared error
    RMSE = math.sqrt(MSE)
    
    #Mean absolute error
    MAE =  sklearn.metrics.mean_absolute_error(y_test, y_pred)
    
    #Median absolute error 
    MdAE = sklearn.metrics.median_absolute_error(y_test, y_pred)
    
    #Mean percentage error
    MAPE = sklearn.metrics.mean_absolute_percentage_error(y_test, y_pred)
    

    
    print('R2: %.3f' % R2)
    print('MSE (Mean squared error): ' f"{MSE:,.0f}")
    print('RMSE (Root mean squared error): ' f"{RMSE:,.0f}")
    print('MAE (Mean absolute error): ' f"{MAE:,.0f}")
    print('MdAE (Median absolute error ): ' f"{MdAE:,.0f}")
    print('MAPE (Mean percentage error): ' f"{MAPE:,.2%}")
   
    
    return

# Prepare and clean data

In [ ]:
# make all columns lowercase
df.columns = df.columns.str.lower()

In [ ]:
df = df[['date','hour','pun']]

In [ ]:
# check if there are nulls
df.info()

In [ ]:
# check values range and basic descriptive statistics
df.describe()

In [ ]:
# there are some hour greater than 24
df[df['hour']  > 24]

In [ ]:
#It is for daylight saving time
#We can delete these rows
# out of range hour filter
df =df[df['hour']  < 25].copy()

In [ ]:
#convert number to time format
df['time'] = df['hour'].apply(num_to_time)

# convert 24 to 0 (midnight)
df['hour'] = df['hour'].apply(lambda x: 0 if x == 24  else x)

In [ ]:
# convert date and time to datetime
df['date_time'] = pd.to_datetime(df.date.astype(str) + ' ' + df.time.astype(str))
df['date_time'] = pd.to_datetime(df['date_time'])

#convert date to date
df['date'] = pd.to_datetime(df['date'])

In [ ]:
# set index and sort ascending
df = df.set_index('date_time')
df = df.sort_index(ascending = True)

In [ ]:
# check the df
df

In [ ]:
# check for duplicates
df.duplicated().sum()

In [ ]:
#Creates time series features from date column
df = create_features(df)


**Plot 30 days Rolling averages**
Rolling averages are useful for finding long-term trends otherwise disguised by occasional fluctuations.

In [ ]:
# 1 month rolling windows plot
rolling = df['pun'].rolling(24*30, center=True).mean()

fig = my_plot(rolling, 'pun',ylabel = "PUN [€/MWh]",title="PUN (Italian Energy Price €/MWh)", navigate = 1)

fig.show()

**Plotting YTD (Year To Date)**

In [ ]:
fig = my_plot(df, 'pun',ylabel = "PUN [€/MWh]",title="PUN (Italian Energy Price €/MWh)", 
              start='2022-01-01', end='2022-06-01')

fig.show()

**Pairplot PUN by Year , Week of Year, Day of Week, Hour**

Pairplot visualizes given data to find the relationship between them where the variables can be continuous or categorical.
Plot pairwise relationships in a data-set.

In [ ]:
# pairplot pun / 'year','month','dayofweek','hour'

sns.pairplot(df,
             hue='hour',
             palette = 'Greens',
             x_vars=['year','month','dayofweek','hour'],
             y_vars='pun',
             height=5,
             plot_kws={'alpha':0.15, 'linewidth':0}
            )

plt.suptitle('PUN by Year , Month , Day of Week, Hour')
plt.show()

**We can see:**
* Decrease of PUN in 2019 (for Covid pandemic an Lockdown)
* Highly increas of pun in 2021 (Due to increse of energy demand after Lockdown) 
* Highly increas of pun in 2021 (Due to increse of energy demand after Lockdown and Russia-Ukraine war)(1)

* There are monthly, weekly and dayli seasonality



(1) Electric Energy price depend also on Gas price. The Italian hub through which gas trading takes place, an electronic system known as the Virtual Exchange Point (PSV) and managed by Snam, a company with public partecipation that deals with the national network of gas pipelines.

VIRTUAL TRADING POINT - Issue and management of gas transactions for balancing the user and the network; display of details and balance for each type of transaction. 

In [ ]:
# filter last three month of data
three_months = 24 * 7 * 13 

#empty dataframe
df_three_months = pd.DataFrame()

# filter backwords
df_three_months = df.iloc[-three_months:].iloc[::-1] 

df_three_months

**Filter data in order to have from Monday to Monday**

In [ ]:
# filter data in order to have from Monday to Monday
df_three_months = from_weekday_to_weekday(df,"Monday")

In [ ]:
# plot 1 day rolling average
df_three_months_roll = df_three_months['pun'].rolling(24, center=True).mean()

fig = my_plot(df_three_months_roll, 'pun',ylabel = "PUN [€/MWh]",title="PUN (Italian Energy Price €/MWh)", navigate = 0)

fig.show()

**Distribution plot**

It is used basically for univariant set of observations and visualizes it through a histogram where Y is the count (or frequency) and X is the varible value (Pun in our case)

In [ ]:
# Pun distribution plot
sns.displot(df_three_months['pun'],bins=30,kde=True,height = 8)

plt.show()

**The distribution is slightly right skewed**

A right-skewed distribution has a right tail. Right-skewed distributions are also called positive-skew distributions. 
That’s because there is a long tail in the positive direction on the number line. The mean is also to the right of the peak.
A right-skewed distribution will have the mean to the right of the median.

**Energy mean PUN per Weekday and Time of the day**

The Hourly Heatmap plots PUN data using a gridded falsecolor map, which displays a data cell for every hour of the week. Hours of day are plotted along the y-axis, and days of year along the x-axis.

In [ ]:
# aggregate on 'dayofweek','dayname','hour' with mean of the PUN
df_three_months_mean = df_three_months[['dayofweek','dayname','hour','pun']].groupby(['dayofweek','dayname', 'hour']).agg({'pun' : 'mean'})
df_three_months_mean = df_three_months_mean.reset_index().pivot(index=['dayofweek','dayname'], columns='hour', values='pun')

ax = plt.subplots(figsize=(15, 8))
plt.title('Energy mean PUN per Weekday and Time of the day')

ax = sns.heatmap(df_three_months_mean,cmap="Blues", 
                  cbar_kws={"orientation": "horizontal"})

ax.set(ylabel='Day Of The Week')

plt.show()

# Dataframe preparation and Train/Test Split

The model requires the dataset to have  2 columns: 'ds' which host the dates and 'y' which hosts the time series values

In [ ]:
df = prepare_df_prophet(df,label = 'pun',date_col = 'date_time', date_index = True)   

In [ ]:
df

**Split train/test**

In [ ]:
# seven days for test
len_test = 24 * 7
#train set
df_train = df.iloc[:-len_test].copy()
#test set
df_test = df.iloc[-len_test:].copy()

# Baseline model and evaluation metrics function

A baseline model is essentially a simple model that acts as a reference in a machine learning project. Its main function is to contextualize the results of trained models. Baseline models are not only very easy to build, but they also provide a lot of information that can dictate the future steps in a machine learning project.

Baseline models serve as benchmarks for trained models

In [ ]:
# baseline Next week is equal to previous week
train_len = len(df_train)
#previus week
forecast = df_train.iloc[train_len-len_test:].copy()
forecast.rename(columns = {'y':'yhat'}, inplace = True)
#shit date 7 days to compare prediction with test
forecast['ds'] =  forecast['ds'] + pd.Timedelta(days = 7)

In [ ]:
#format forecast and test
y_test,y_pred = format_fore_test_df(df_test,forecast)
# plot prediction vs actual
plot_pred_test(y_test,y_pred)  

In [ ]:
# calculate error metrics
error_metrics(y_test, y_pred)

9.21% Our baseline model is not bad!

# Prophet model

![](https://miro.medium.com/max/1400/1*sjUBCoQXy5rfWOfguP_K1A.png)

# Simple prophet model

In [ ]:
# Setup the model with no customization
model = Prophet()

# fit the model
model.fit(df_train)

future = model.make_future_dataframe(periods=len_test,freq='H')

forecast = model.predict(future)

In [ ]:
forecast

In [ ]:
y_test,y_pred = format_fore_test_df(df_test,forecast)

In [ ]:
plot_pred_test(y_test,y_pred)  

In [ ]:
error_metrics(y_test, y_pred)

**We can do really better! The model does not seams to capure the trend and it is underfitted. It is worst tha the baseline**

We see from pun trend, that last year we have had a strong changenging in trend that the previous model did not capture  

# Prophet model with custum parameters

Prophet includes functionality for time series **cross validation** to measure forecast error using historical data. This is done by selecting cutoff points in the history, and for each of them fitting the model using data only up to that cutoff point. We can then compare the forecasted values to the actual values. This figure illustrates a simulated historical forecast on the Peyton Manning dataset, where the model was fit to a initial history of 5 years, and a forecast was made on a one year horizon.


![](https://facebook.github.io/prophet/static/diagnostics_files/diagnostics_4_0.png)

In [ ]:
# My parameter grid to optimize the model
import itertools
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
     # Fourier Order
    'weekly_seasonality': [5,7,9],
    'daily_seasonality': [4,6,8]
}


**After some hours the cross-validation has given to me the following best model hyperparameters:**

In [ ]:
# instantiate the model

model = Prophet(
# additive or multiplicative     
seasonality_mode='multiplicative',
# Fourier Order for Seasonalities    
weekly_seasonality=9,
daily_seasonality= 8,
yearly_seasonality= True,
# holiday effect
holidays_prior_scale=0.05,
# changepoint effect
changepoint_prior_scale=0.1,
# seasonality scale effect     
seasonality_prior_scale=10,
# to speed up the process without having the confidence interval values
uncertainty_samples=0,
# By default changepoints are only inferred for the first 80% of the time series in order 
#to have plenty of runway for projecting the trend forward and to avoid overfitting fluctuations at the end of the time series.     
# We see from trend that last period we have had a strong changenging in trend that the previous model did not capture    
changepoint_range=.9

)
# add holidays
model.add_country_holidays(country_name='IT')
# fit model
model.fit(df_train)
#forecast
future = model.make_future_dataframe(periods=len_test,freq='H')
forecast = model.predict(future)

In [ ]:
# format test and forecast
y_test,y_pred = format_fore_test_df(df_test,forecast)

In [ ]:
# plot test and forecast
plot_pred_test(y_test,y_pred)  

In [ ]:
# error metrics
error_metrics(y_test, y_pred)

**MAPE (Mean percentage error): 6.82%** We have improved our model a lot

In [ ]:
# plot changepoint, notice the trend change after half of 2021
from prophet.plot import add_changepoints_to_plot
fig = model.plot(forecast)
add_changepoints_to_plot(fig.gca(), model, forecast)
plt.show()

If you want to see the forecast components, you can use the Prophet.plot_components method. By default you’ll see the trend, yearly seasonality, and weekly seasonality of the time series. If you include holidays, you’ll see those here, too.

In [ ]:
fig1 = model.plot_components(forecast)